In [1]:
#pip install -e .

In [2]:
from configparser import ConfigParser
from td.credentials import TdCredentials
from td.client import TdAmeritradeClient

import inspect

import pandas as pd
from pprint import pprint


In [3]:
# Permanent parameters:

config_file='config/config.ini'
credential_file='config/td_credentials.json'

# Section 1: Authorization

In [4]:
# Initialize the Parser.
config = ConfigParser()

In [5]:
# Read the file.
config.read(config_file)

['config/config.ini']

In [6]:
# Get the specified credentials.
client_id = config.get('main', 'client_id')
redirect_uri = config.get('main', 'redirect_uri')

In [7]:
redirect_uri

'http://127.0.0.1'

In [8]:

# Intialize our `Crednetials` object.
td_credentials = TdCredentials(
    
    client_id=client_id,

    redirect_uri=redirect_uri,

    credential_file=credential_file    # just in case the file exists, it won't do authorizatio again. 
)

In [9]:
# Saving the credential JSON file:
td_credentials.to_credential_file(
    file_path=credential_file
)

In [10]:
# Initalize the `TdAmeritradeClient`
td_client = TdAmeritradeClient(
    credentials=td_credentials
)

# use_quotes

In [11]:
# Initialize the `Quotes` service.
quote_service = td_client.quotes()

In [12]:
# Grab a single quote.
print(
    quote_service.get_quote(instrument='AAPL')
)

{'AAPL': {'assetType': 'EQUITY', 'assetMainType': 'EQUITY', 'cusip': '037833100', 'assetSubType': '', 'symbol': 'AAPL', 'description': 'Apple Inc. - Common Stock', 'bidPrice': 143.8, 'bidSize': 6100, 'bidId': 'P', 'askPrice': 143.85, 'askSize': 300, 'askId': 'P', 'lastPrice': 143.8, 'lastSize': 0, 'lastId': 'D', 'openPrice': 142.7, 'highPrice': 144.34, 'lowPrice': 142.28, 'bidTick': ' ', 'closePrice': 143.0, 'netChange': 0.8, 'totalVolume': 65874459, 'quoteTimeInLong': 1675213183986, 'tradeTimeInLong': 1675213198390, 'mark': 143.85, 'exchange': 'q', 'exchangeName': 'NASD', 'marginable': True, 'shortable': True, 'volatility': 0.0132, 'digits': 4, '52WkHigh': 179.61, '52WkLow': 124.17, 'nAV': 0.0, 'peRatio': 23.9147, 'divAmount': 0.92, 'divYield': 0.64, 'divDate': '2022-11-04 00:00:00.000', 'securityStatus': 'Normal', 'regularMarketLastPrice': 144.29, 'regularMarketLastSize': 146208, 'regularMarketNetChange': 1.29, 'regularMarketTradeTimeInLong': 1675198801523, 'netPercentChangeInDouble'

### Success!

## Price History

In [13]:
from td.utils.enums import PeriodType
from td.utils.enums import FrequencyType


from datetime import datetime
from datetime import timedelta

In [14]:
# Initialize the `PriceHistory` service.
price_history_service = td_client.price_history()

In [15]:
# Grab the Price History, with enums.
price_history = price_history_service.get_price_history(
    symbol='MSFT',
    frequency_type=FrequencyType.Minute,
    frequency=1,
    period_type=PeriodType.Day,
    period=10,
    extended_hours_needed=False
)

In [16]:
# The max look back period for minute data is 31 Days.
end_date = datetime.now()
start_date = datetime.now() - timedelta(days=60)

In [17]:
# Grab the Price History, custom time frame.
price_history = price_history_service.get_price_history(
    symbol='MSFT',
    frequency_type=FrequencyType.Minute,
    frequency=1,
    start_date=start_date,
    end_date=end_date,
    extended_hours_needed=False
)

In [18]:
#price_history

In [19]:
# print(price_history['candles'])

# use movers

In [20]:
# Initialize the `Movers` service.
movers_service = td_client.movers()

# Grab the top 10 percentage up movers.
pprint(
    movers_service.get_movers(
        index='$DJI',
        direction='down',
        change='percent'
    )
)


[{'change': -0.03521988527724668,
  'description': 'Caterpillar, Inc. Common Stock',
  'direction': 'down',
  'last': 252.29,
  'symbol': 'CAT',
  'totalVolume': 15929517},
 {'change': -0.012883458230278007,
  'description': "McDonald's Corporation Common Stock",
  'direction': 'down',
  'last': 267.4,
  'symbol': 'MCD',
  'totalVolume': 8102062},
 {'change': -0.004212860310421445,
  'description': 'International Business Machines Corporation Common Stock',
  'direction': 'down',
  'last': 134.73,
  'symbol': 'IBM',
  'totalVolume': 13729472},
 {'change': -0.0010332950631456856,
  'description': 'Chevron Corporation Common Stock',
  'direction': 'down',
  'last': 174.02,
  'symbol': 'CVX',
  'totalVolume': 27322174}]


# Use orders

In [21]:
from td.utils.enums import OrderStatus

In [22]:
# This piece of info from the config file was not necessary until this time:
account_number = config.get('main', 'account_number')

In [23]:
# Initialize the `Orders` service.
orders_service = td_client.orders()

# Query all our orders for a specific account.
pprint(
    orders_service.get_orders_by_path(
        account_id=account_number,
        order_status=OrderStatus.Expired
        #order_status=OrderStatus.Queded
    )
)

[]


In [24]:
# Query all our orders.
pprint(
    orders_service.get_orders_by_query()
)

[]


BIG HINT: We can set the BEGIN and END values for the order history, see the code. 

# option chain

In [25]:

from td.utils.enums import OptionaRange
from td.utils.enums import OptionType
from td.utils.enums import ContractType
from td.utils.enums import ExpirationMonth
from td.utils.option_chain import OptionChainQuery


In [26]:
# Initialize the `OptionsChain` service.
options_chain_service = td_client.options_chain()

In [27]:
# Method 1: Build a Query using the `OptionChainQuery` object.
# This method is preferred because I'll check some of your inputs
# to make sure you are sending the correct parameters.
option_chain_query = OptionChainQuery(
    symbol='MSFT',
    contract_type=ContractType.Call,
    expiration_month=ExpirationMonth.June,
    option_type=OptionType.StandardContracts,
    option_range=OptionaRange.InTheMoney,
    include_quotes=True
)

In [28]:
# Query the Options Data.

# Big hint: it seems that it cannot be run twice!
options_data = options_chain_service.get_option_chain(
    option_chain_query=option_chain_query
)

pprint(options_data['numberOfContracts'])

930


The original repo suggests a second methos, which is exactly the same as this one, but instead of using utils, the dictionary is made manually. I'm not bringing that. 

# Use streaming client

In [29]:
from td.utils.enums import LevelOneQuotes
from td.utils.enums import LevelOneOptions
from td.utils.enums import LevelOneFutures
from td.utils.enums import LevelOneForex
from td.utils.enums import LevelOneFuturesOptions
from td.utils.enums import NewsHeadlines
from td.utils.enums import ChartServices
from td.utils.enums import ChartEquity
from td.utils.enums import TimesaleServices
from td.utils.enums import Timesale
from td.utils.enums import ActivesServices
from td.utils.enums import ActivesVenues
from td.utils.enums import ActivesDurations
from td.utils.enums import ChartFuturesFrequencies
from td.utils.enums import ChartFuturesPeriods
from td.utils.enums import LevelTwoQuotes
from td.utils.enums import LevelTwoOptions

In [30]:
# Initialize the `StreamingApiClient` service.
streaming_api_service = td_client.streaming_api_client()

In [31]:
# Let's see what services we have access to.
streaming_services = streaming_api_service.services()

In [32]:
# Set the Quality of Service.
streaming_services.quality_of_service(
    qos_level='1'
)

In [33]:
# Grab level one quotes.
streaming_services.level_one_quotes(
    symbols=['MSFT'],
    fields=LevelOneQuotes.All
)

In [34]:
# Grab level one options quotes.
streaming_services.level_one_options(
    symbols=['MSFT_043021C120'],
    fields=LevelOneOptions.All
)

In [35]:
# Grab level one futures quotes.
streaming_services.level_one_futures(
    symbols=['/ESM4', '/ES'],
    fields=LevelOneFutures.All
)


In [36]:
# Grab level one forex quotes.
streaming_services.level_one_forex(
    symbols=['EUR/USD'],
    fields=LevelOneForex.All
)

In [37]:
# Stream News Headlines.
streaming_services.news_headline(
    symbols=['MSFT', 'GOOG', 'AAPL'],
    fields=NewsHeadlines.All
)


In [38]:
# Stream Level One Futures Options.
streaming_services.level_one_futures_options(
    symbols=['./CLM21P625'],
    fields=LevelOneFuturesOptions.All
)


In [39]:
# Stream equity bars.
streaming_services.chart(
    service=ChartServices.ChartEquity,
    symbols=['MSFT', 'GOOG', 'AAPL'],
    fields=ChartEquity.All
)

In [40]:
# Stream Time & Sales data.
streaming_services.timesale(
    service=TimesaleServices.TimesaleEquity,
    symbols=['MSFT', 'GOOG', 'AAPL'],
    fields=Timesale.All
)

In [41]:
# Stream the Actives.
streaming_services.actives(
    service=ActivesServices.ActivesNasdaq,
    venue=ActivesVenues.NasdaqExchange,
    duration=ActivesDurations.All
)

In [42]:
# Stream Historical Futures Prices.
streaming_services.chart_history_futures(
    symbols=['/ES', '/CL'],
    frequency=ChartFuturesFrequencies.OneMinute,
    period=ChartFuturesPeriods.OneDay
)

In [43]:
# Stream Level Two Quotes.
streaming_services.level_two_quotes(
    symbols=['MSFT', 'PINS'],
    fields=LevelTwoQuotes.All
)

In [44]:
# Stream Level Two Quotes.
streaming_services.level_two_options(
    symbols=['MSFT_043021C120'],
    fields=LevelTwoOptions.All
)

In [45]:
# Start Streaming.

# Hint: this does not work in a notebook!

#streaming_api_service.open_stream()

Can't run it in a notebook. I tested the code: "./samples/temp_use_streaming_client.py" which I modified by changing the '../config/config.ini' 

# Data pipeline

In [59]:

async def data_pipeline():
    """
    This is an example of how to build a data pipeline,
    using the library. A common scenario that would warrant
    using a pipeline is taking data that is sent back to the stream
    and processing it so it can be used in other programs or functions.

    Generally speaking, you will need to wrap the operations that process
    and handle the data inside an async function. The reason being is so
    that you can await the return of data.

    However, operations like creating the client, and subscribing to services
    can be performed outside the async function. In the example below, we demonstrate
    building the pipline, which is connecting to the websocket and logging in.

    We then start the pipeline, which is where the services are subscribed to and data
    begins streaming. The `start_pipeline()` will return the data as it comes in. From
    there, we process the data however we choose.

    Additionally, we can also see how to unsubscribe from a stream using logic and how
    to close the socket mid-stream.
    """

    data_response_count = 0
    heartbeat_response_count = 0

    # Build the Pipeline.
    await streaming_api_service.build_pipeline()

    # Keep going as long as we can recieve data.
    while True:

        # Start the Pipeline.
        data = await streaming_api_service.start_pipeline()

        # Grab the Data, if there was any. Remember not every message will have `data.`
        if data and 'data' in data:

            print('='*80)

            data_content = data['data'][0]['content']
            pprint(data_content, indent=4)

            # Here I can grab data as it comes in and do something with it.
            if 'key' in data_content[0]:
                print('Here is my key: {}'.format(data_content[0]['key']))

            print('-'*80)
            data_response_count += 1

        # If we get a heartbeat notice, let's increment our counter.
        elif data and 'notify' in data:
            print(data['notify'][0])
            heartbeat_response_count += 1

        # Once we have 1 data responses, we can unsubscribe from a service.
        if data_response_count == 1:
            unsub = await streaming_api_service.unsubscribe(service='LEVELONE_QUOTES')
            data_response_count += 1
            print('='*80)
            print(unsub)
            print('-'*80)

        # Once we have 5 heartbeats, let's close the stream. Make sure to break the while loop.
        # or else you will encounter an exception.
        if heartbeat_response_count == 3:
            await streaming_api_service.close_stream()
            break

        heartbeat_response_count += 1


In [62]:
#data_pipeline()

# use accounts

In [47]:
# Initialize the `Accounts` service.
accounts_service = td_client.accounts()


In [48]:
# Grab all the Positions and Orders for a specific account.
pprint(
    accounts_service.get_accounts(
        account_id=account_number,
        include_orders=True,
        include_positions=True
    )
)

Access Token Expired, refreshing access token...
{'securitiesAccount': {'accountId': '253157675',
                       'currentBalances': {'accruedInterest': 0.0,
                                           'availableFunds': 2175.74,
                                           'availableFundsNonMarginableTrade': 2175.74,
                                           'bondValue': 0.0,
                                           'buyingPower': 4351.48,
                                           'buyingPowerNonMarginableTrade': 2175.74,
                                           'cashBalance': 1250.21,
                                           'cashReceipts': 0.0,
                                           'dayTradingBuyingPower': 0.0,
                                           'equity': 2973.16,
                                           'equityPercentage': 100.0,
                                           'liquidationValue': 2973.16,
                                           'longMarginVa

In [51]:
from td.utils.enums import TransactionTypes
# Grab all the Positions and Orders for all my accounts.
pprint(
    accounts_service.get_accounts(
        include_orders=True,
        include_positions=True
    )
)


[{'securitiesAccount': {'accountId': '253157675',
                        'currentBalances': {'accruedInterest': 0.0,
                                            'availableFunds': 2175.74,
                                            'availableFundsNonMarginableTrade': 2175.74,
                                            'bondValue': 0.0,
                                            'buyingPower': 4351.48,
                                            'buyingPowerNonMarginableTrade': 2175.74,
                                            'cashBalance': 1250.21,
                                            'cashReceipts': 0.0,
                                            'dayTradingBuyingPower': 0.0,
                                            'equity': 2973.16,
                                            'equityPercentage': 100.0,
                                            'liquidationValue': 2973.16,
                                            'longMarginValue': 1722.95,
                    

In [52]:
# Grab all the transactions for a specific account.
pprint(
    accounts_service.get_transactions(
        account_id=account_number,
        transaction_type=TransactionTypes.All
    )
)

[{'cashBalanceEffectFlag': True,
  'description': 'FREE BALANCE INTEREST ADJUSTMENT',
  'fees': {'additionalFee': 0.0,
           'cdscFee': 0.0,
           'commission': 0.0,
           'optRegFee': 0.0,
           'otherCharges': 0.0,
           'rFee': 0.0,
           'regFee': 0.0,
           'secFee': 0.0},
  'netAmount': 0.74,
  'settlementDate': '2022-12-30',
  'subAccount': '2',
  'transactionDate': '2022-12-31T04:36:27+0000',
  'transactionId': 47309111956,
  'transactionItem': {'accountId': 253157675, 'cost': 0.0},
  'transactionSubType': 'CA',
  'type': 'DIVIDEND_OR_INTEREST'},
 {'cashBalanceEffectFlag': True,
  'description': 'BUY TRADE',
  'fees': {'additionalFee': 0.0,
           'cdscFee': 0.0,
           'commission': 0.0,
           'optRegFee': 0.0,
           'otherCharges': 0.0,
           'rFee': 0.0,
           'regFee': 0.0,
           'secFee': 0.0},
  'netAmount': -1848.55,
  'orderDate': '2022-12-24T23:34:23+0000',
  'orderId': '10020487227',
  'settlementDate

In [53]:
# Grab a specific transaction for a specific account.
pprint(
    accounts_service.get_transaction(
        account_id=account_number,
        transaction_id='27444883992'
    )
)


HTTPError: 

# User info:

In [54]:
# Initialize the `UserInfo` service.
user_info_service = td_client.user_info()

# Grab the preferences for a specific account.
pprint(
    user_info_service.get_preferences(
        account_id=account_number
    )
)

{'authTokenTimeout': 'FIFTY_FIVE_MINUTES',
 'defaultAdvancedToolLaunch': 'NONE',
 'defaultEquityOrderDuration': 'DAY',
 'defaultEquityOrderLegInstruction': 'NONE',
 'defaultEquityOrderMarketSession': 'NORMAL',
 'defaultEquityOrderPriceLinkType': 'NONE',
 'defaultEquityOrderType': 'LIMIT',
 'defaultEquityQuantity': 0,
 'directEquityRouting': False,
 'directOptionsRouting': False,
 'equityTaxLotMethod': 'FIFO',
 'expressTrading': False,
 'mutualFundTaxLotMethod': 'FIFO',
 'optionTaxLotMethod': 'FIFO'}


In [55]:
# Grab the streamer subscription keys.
pprint(
    user_info_service.get_streamer_subscription_keys(
        account_ids=[account_number]
    )
)


{'keys': [{'key': 'b4faa841ff618cc3bbd8c4757203a7f4a3d932548275e91aad271d5e14d28e2fa9fb48daf986e2429d69ed85384efcb09'}]}


In [56]:
# Grab User Principals.
pprint(
    user_info_service.get_user_principals()
)

{'accessLevel': 'CUS',
 'accounts': [{'accountCdDomainId': 'A000000094724983',
               'accountId': '253157675',
               'acl': 'BPDRDTDWESF7G1G3G5G7GKGLH1H3H5LTM1MAOCPNQSRFSDT5TETFTOTTUAURXBXNXO',
               'authorizations': {'advancedMargin': True,
                                  'apex': False,
                                  'autoPositionEffect': False,
                                  'levelTwoQuotes': False,
                                  'marginTrading': True,
                                  'optionTradingLevel': 'COVERED',
                                  'scottradeAccount': False,
                                  'stockTrading': True,
                                  'streamerAccess': True,
                                  'streamingNews': False},
               'company': 'AMER',
               'displayName': 'mehdishaf',
               'preferences': {'authTokenTimeout': 'FIFTY_FIVE_MINUTES',
                               'defaultAdvancedTool

In [57]:
# Method 1, Update the User Preferences.
user_info_service.update_user_preferences(
    account_id=account_number,
    preferences={
        'authTokenTimeout': 'EIGHT_HOURS',
        'defaultAdvancedToolLaunch': 'NONE',
        'defaultEquityOrderDuration': 'DAY',
        'defaultEquityOrderLegInstruction': 'NONE',
        'defaultEquityOrderMarketSession': 'NORMAL',
        'defaultEquityOrderPriceLinkType': 'NONE',
        'defaultEquityOrderType': 'LIMIT',
        'defaultEquityQuantity': 0,
        'equityTaxLotMethod': 'FIFO',
        'expressTrading': True,
        'mutualFundTaxLotMethod': 'FIFO',
        'optionTaxLotMethod': 'FIFO'
    }
)

# Using watchlists

In [63]:

# Initialize the `Watchlists` service.
watchlists_service = td_client.watchlists()


In [64]:

# Grab all watchlists for all accounts.
pprint(
    watchlists_service.get_all_accounts_watchlists()
)

Access Token Expired, refreshing access token...
[{'accountId': '253157675',
  'name': 'Indexes',
  'watchlistId': '1882557907',
  'watchlistItems': [{'averagePrice': 0.0,
                      'commission': 0.0,
                      'instrument': {'assetType': 'INDEX', 'symbol': '$DJI'},
                      'quantity': 0.0,
                      'sequenceId': 1},
                     {'averagePrice': 0.0,
                      'commission': 0.0,
                      'instrument': {'assetType': 'EQUITY', 'symbol': 'SPX'},
                      'quantity': 0.0,
                      'sequenceId': 2},
                     {'averagePrice': 0.0,
                      'commission': 0.0,
                      'instrument': {'assetType': 'EQUITY',
                                     'symbol': 'COMP:GIDS'},
                      'quantity': 0.0,
                      'sequenceId': 3},
                     {'averagePrice': 0.0,
                      'commission': 0.0,
                     

In [66]:

# Grab a specific watchlist for a specific account.
pprint(
    watchlists_service.get_watchlist(
        account_id=account_number,
        watchlist_id='1882557907'
    )
)

{'accountId': '253157675',
 'name': 'Indexes',
 'watchlistId': '1882557907',
 'watchlistItems': [{'averagePrice': 0.0,
                     'commission': 0.0,
                     'instrument': {'assetType': 'INDEX', 'symbol': '$DJI'},
                     'quantity': 0.0,
                     'sequenceId': 1},
                    {'averagePrice': 0.0,
                     'commission': 0.0,
                     'instrument': {'assetType': 'EQUITY', 'symbol': 'SPX'},
                     'quantity': 0.0,
                     'sequenceId': 2},
                    {'averagePrice': 0.0,
                     'commission': 0.0,
                     'instrument': {'assetType': 'EQUITY',
                                    'symbol': 'COMP:GIDS'},
                     'quantity': 0.0,
                     'sequenceId': 3},
                    {'averagePrice': 0.0,
                     'commission': 0.0,
                     'instrument': {'assetType': 'EQUITY', 'symbol': 'VIX'},
              

In [72]:
# Grab all the watchlists for a specific account
pprint(
    watchlists_service.get_accounts_watchlist(
        account_id=account_number
    )
)

[{'accountId': '253157675',
  'name': 'space companies',
  'watchlistId': '2126167472',
  'watchlistItems': [{'averagePrice': 0.0,
                      'commission': 0.0,
                      'instrument': {'assetType': 'EQUITY', 'symbol': 'SPCE'},
                      'quantity': 0.0,
                      'sequenceId': 1},
                     {'averagePrice': 0.0,
                      'commission': 0.0,
                      'instrument': {'assetType': 'EQUITY', 'symbol': 'UFO'},
                      'quantity': 0.0,
                      'sequenceId': 2}]},
 {'accountId': '253157675',
  'name': 'Quotes',
  'watchlistId': '1882557909',
  'watchlistItems': [{'averagePrice': 0.0,
                      'commission': 0.0,
                      'instrument': {'assetType': 'EQUITY', 'symbol': 'IBM'},
                      'quantity': 0.0,
                      'sequenceId': 1},
                     {'averagePrice': 0.0,
                      'commission': 0.0,
                      '

In [68]:
# Define some items for our watchlist.
watchlist_items = [
    {
        'quantity': 0.0,
        'averagePrice': 0.0,
        'commission': 0.0,
        'instrument': {
            'symbol': 'SPCE',
            'assetType': 'EQUITY'
        }
    },
    {
        'quantity': 0.0,
        'averagePrice': 0.0,
        'commission': 0.0,
        'instrument': {
            'symbol': 'UFO',
            'assetType': 'EQUITY'
        }
    }
]

In [69]:
watchlist_items

[{'quantity': 0.0,
  'averagePrice': 0.0,
  'commission': 0.0,
  'instrument': {'symbol': 'SPCE', 'assetType': 'EQUITY'}},
 {'quantity': 0.0,
  'averagePrice': 0.0,
  'commission': 0.0,
  'instrument': {'symbol': 'UFO', 'assetType': 'EQUITY'}}]

In [70]:
# Create a new watchlist.
pprint(
    watchlists_service.create_watchlist(
        account_id=account_number,
        name='space companies',
        watchlist_items=watchlist_items
    )
)

None


In [73]:
# Update a watchlist.
pprint(
    watchlists_service.update_watchlist(
        account_id=account_number,
        watchlist_id='253157675',
        name='Space_Companies_Current',
        watchlist_items=[
            {
                'quantity': 0.0,
                'averagePrice': 0.0,
                'commission': 0.0,
                'instrument': {
                    'symbol': 'TSLA',
                    'assetType': 'EQUITY'
                },
                'sequenceId': 3
            }
        ]
    )
)


HTTPError: 

# Use instruments
Search for specific instruments

In [75]:
# Initialize the `Instruments` service.
instruments_service = td_client.instruments()

# Search for a symbol.
pprint(
    instruments_service.search_instruments(
        symbol='MSFT',
        projection='symbol-search'
    )
)


{'MSFT': {'assetType': 'EQUITY',
          'cusip': '594918104',
          'description': 'Microsoft Corporation - Common Stock',
          'exchange': 'NASDAQ',
          'symbol': 'MSFT'}}


In [77]:
from td.utils.enums import Projections
# Search for companies using description regular expression.
pprint(
    instruments_service.search_instruments(
        symbol='[Quantum Computing]',
        projection=Projections.DescriptionRegex
    )
)

{}


# Price History


In [78]:
# Initialize the `PriceHistory` service.
price_history_service = td_client.price_history()

# Grab the Price History, with enums.
price_history = price_history_service.get_price_history(
    symbol='MSFT',
    frequency_type=FrequencyType.Minute,
    frequency=1,
    period_type=PeriodType.Day,
    period=10,
    extended_hours_needed=False
)


In [79]:
pprint(price_history)

{'candles': [{'close': 234.9,
              'datetime': 1674138600000,
              'high': 234.98,
              'low': 233.78,
              'open': 233.78,
              'volume': 455341},
             {'close': 234.36,
              'datetime': 1674138660000,
              'high': 234.98,
              'low': 234.3279,
              'open': 234.88,
              'volume': 94944},
             {'close': 235.03,
              'datetime': 1674138720000,
              'high': 235.52,
              'low': 234.26,
              'open': 234.37,
              'volume': 147931},
             {'close': 235.13,
              'datetime': 1674138780000,
              'high': 235.2199,
              'low': 234.77,
              'open': 235.03,
              'volume': 87306},
             {'close': 234.73,
              'datetime': 1674138840000,
              'high': 235.14,
              'low': 234.63,
              'open': 235.11,
              'volume': 79007},
             {'close': 234.17,

In [90]:
# The max look back period for minute data is 31 Days.
end_date = datetime.now()
start_date = datetime.now() - timedelta(days=60)

# Grab the Price History, custom time frame.
price_history = price_history_service.get_price_history(
    symbol='MSFT',
    #frequency_type=FrequencyType.Minute,
    frequency_type= FrequencyType.Weekly,
    period_type= 'year',
    frequency=1,
    start_date=start_date,
    end_date=end_date,
    extended_hours_needed=False
)
print(price_history['candles'])

[{'open': 252.01, 'high': 253.819, 'low': 242.205, 'close': 245.42, 'volume': 109602043, 'datetime': 1670220000000}, {'open': 247.445, 'high': 263.915, 'low': 243.51, 'close': 244.69, 'volume': 229942973, 'datetime': 1670824800000}, {'open': 244.86, 'high': 245.615, 'low': 233.87, 'close': 238.73, 'volume': 128396491, 'datetime': 1671429600000}, {'open': 238.7, 'high': 241.92, 'low': 234.17, 'close': 239.82, 'volume': 75854917, 'datetime': 1672034400000}, {'open': 243.08, 'high': 245.75, 'low': 219.35, 'close': 224.93, 'volume': 159562627, 'datetime': 1672639200000}, {'open': 226.45, 'high': 239.9, 'low': 226.41, 'close': 239.23, 'volume': 131675747, 'datetime': 1673244000000}, {'open': 237.97, 'high': 242.38, 'low': 230.68, 'close': 240.22, 'volume': 123872791, 'datetime': 1673848800000}, {'open': 241.1, 'high': 249.83, 'low': 230.9, 'close': 248.16, 'volume': 198648466, 'datetime': 1674453600000}, {'open': 244.51, 'high': 264.69, 'low': 242.2, 'close': 264.6, 'volume': 123608786, 'da